In [1]:
import pandas as pd
filename='80k_articles.txt'
all_content=open(filename,encoding='utf-8').read()
all_content[:200]

'新华社照片，东莞（广东），2017年4月7日\\n（体育）（9）篮球——CBA总决赛第四场：广东对阵新疆\\n4月7日，广东东莞银行队球员易建联在比赛中扣篮。\\n当日，在2016-2017赛季中国男子篮球职业联赛（CBA）总决赛第四场比赛中，广东东莞银行队主场迎战新疆喀什古城队。\\n新华社记者孟永民摄\\n\u3000\u3000新华社北京４月１４日新媒体专电（记者杨烨）作为国民经济的重要支柱，央企一季度交上了一份漂亮的“'

# 正则测试

In [2]:
import requests
import re
text=requests.get('https://movie.douban.com/').text
re.findall('<img src="(.*?)"',text)[1]

'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2539424621.jpg'

# 数据清理

In [3]:
def tokenize(string):
    str_t=''.join(re.findall('[\w|\d]+',string))
    str_temp=str_t.replace('\u3000','')
    return str_temp.replace('\\n','').replace('n','')

In [4]:
tokenize(all_content[:200])

'新华社照片东莞广东2017年4月7日体育9篮球CBA总决赛第四场广东对阵新疆4月7日广东东莞银行队球员易建联在比赛中扣篮当日在20162017赛季中国男子篮球职业联赛CBA总决赛第四场比赛中广东东莞银行队主场迎战新疆喀什古城队新华社记者孟永民摄新华社北京４月１４日新媒体专电记者杨烨作为国民经济的重要支柱央企一季度交上了一份漂亮的'

In [5]:
ALL_CHARACTER=tokenize(all_content)
len(ALL_CHARACTER)

29128532

# Unigram

In [6]:
import jieba

In [7]:
def sentence_seg(string):
    str_temp=tokenize(string)
    word_list=jieba.lcut(str_temp,cut_all=False)
    return word_list

In [8]:
word_seg=sentence_seg(ALL_CHARACTER)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lujun\AppData\Local\Temp\jieba.cache
Loading model cost 0.743 seconds.
Prefix dict has been built succesfully.


In [9]:
##统计每个词出现的概率为计算条件概率做准备
from collections import Counter
all_character_counts=Counter(word_seg)

In [10]:
all_character_counts

Counter({'新华社': 127277,
         '照片': 46662,
         '东莞': 1401,
         '广东': 3853,
         '2017': 64556,
         '年': 109758,
         '4': 69004,
         '月': 169019,
         '7': 12286,
         '日': 147372,
         '体育': 22112,
         '9': 10024,
         '篮球': 4268,
         'CBA': 883,
         '总决赛': 1961,
         '第四场': 416,
         '对阵': 2127,
         '新疆': 4205,
         '银行': 5456,
         '队': 12584,
         '球员': 12950,
         '易建联': 105,
         '在': 234823,
         '比赛': 28811,
         '中': 65916,
         '扣篮': 116,
         '当日': 24276,
         '20162017': 6170,
         '赛季': 8637,
         '中国': 63390,
         '男子篮球': 391,
         '职业联赛': 463,
         '主场': 5923,
         '迎战': 618,
         '喀什': 835,
         '古城': 1389,
         '记者': 55624,
         '孟永民摄': 117,
         '北京': 22561,
         '４': 30775,
         '１': 78756,
         '日新': 2075,
         '媒体': 8718,
         '专电': 2122,
         '杨烨': 15,
         '作为': 9301,
         '国

In [11]:
from matplotlib.pyplot import yscale, xscale, title, plot
M=all_character_counts.most_common()[0][1]
yscale('log'); xscale('log'); title('Frequency of n-th most frequent word and 1/n line.')
plot([c for (w, c) in all_character_counts.most_common()])
plot([M/i for i in range(1, len(all_character_counts)+1)]);

In [12]:
def get_prob_from_counts(count):
    sum_counts=sum(count.values())
    def get_word_prob(item):
        return (count[item]+1)/(sum_counts+1)
    return get_word_prob
get_word_prob=get_prob_from_counts(all_character_counts)

#每次求概率前都要计算一次总和，速度变慢
def get_word_probability(char):
    return (all_character_counts[char]+1)/(sum(all_character_counts.values())+1)

# 检验运行速度

In [13]:
import time

In [14]:
def get_running_time(func,arg,times):
    strat_time=time.time()
    for i in range(times):
        func(arg)
    print('{}运行的时间是{}'.format(func.__name__,time.time()-strat_time))

In [17]:
get_running_time(get_word_prob,'喜欢',10000)

get_word_prob运行的时间是0.0029761791229248047


In [18]:
get_running_time(get_word_probability,'喜欢',10000)

get_word_probability运行的时间是22.974759578704834


In [19]:
from functools import reduce
from operator import add,mul

In [20]:
reduce(add,range(1,1000))

499500

In [21]:
def get_prob(string):
    return reduce(mul,[get_word_prob(e) for e in sentence_seg(string)])

In [22]:
get_prob('这是一个比较常见测试用例')##可能是其中有某个词没有出现过

1.5874825891822571e-22

In [23]:
get_prob('这是一个比较罕见的测试用例')

7.658013060381697e-24

In [24]:
pair = """前天晚上吃晚饭的时候
前天晚上吃早饭的时候""".split('\n')

pair2 = """正是一个好看的小猫
真是一个好看的小猫""".split('\n')

pair3 = """我无言以对，简直
我简直无言以对""".split('\n')

In [25]:
pairs=[pair,pair2,pair3]
pairs

[['前天晚上吃晚饭的时候', '前天晚上吃早饭的时候'],
 ['正是一个好看的小猫', '真是一个好看的小猫'],
 ['我无言以对，简直', '我简直无言以对']]

In [26]:
def get_prob_performance(func,strings):
    for m,n in pairs:
        print('{}出现的概率是{}'.format(m,func(m)))
        print('{}出现的概率是{}'.format(n,func(n)))

In [27]:
get_prob_performance(get_prob,pairs)

前天晚上吃晚饭的时候出现的概率是4.041843554119873e-22
前天晚上吃早饭的时候出现的概率是2.3217538029146583e-25
正是一个好看的小猫出现的概率是1.1533767205925158e-19
真是一个好看的小猫出现的概率是2.2018228223093513e-20
我无言以对，简直出现的概率是8.713901812802775e-16
我简直无言以对出现的概率是8.713901812802775e-16


# 2-gram

In [29]:
###[word_seg[i:i+2] for i in range(len(word_seg)-1)] 内存不够
word_seg=word_seg[:1500000]
word_seg_new=[word_seg[i:i+2] for i in range(len(word_seg)-1)]

In [32]:
def two_gram_word_counter(word_seg_new):
    two_gram_counter=[]
    for i in word_seg_new:
        word_new=''.join(i)
        two_gram_counter.append(word_new)
    return Counter(two_gram_counter)

In [33]:
two_gram_counts=two_gram_word_counter(word_seg_new)

In [36]:
two_gram_counts.most_common()

[('外代二线', 5952),
 ('2017年', 5575),
 ('5月', 5076),
 ('4月', 5027),
 ('新华社照片', 4391),
 ('新华社记者', 2555),
 ('２０', 2336),
 ('日在', 2307),
 ('年5', 2187),
 ('年4', 2165),
 ('日外代', 1842),
 ('照片外代', 1840),
 ('外代2017', 1815),
 ('比赛中', 1784),
 ('6月', 1732),
 ('一带一路', 1603),
 ('０１', 1550),
 ('００', 1503),
 ('当日在', 1427),
 ('摄新华社', 1241),
 ('完新华社', 1153),
 ('日体育', 1092),
 ('二线足球', 1086),
 ('１０', 1085),
 ('日电记者', 1007),
 ('举行的', 852),
 ('在比赛', 847),
 ('2016年', 755),
 ('二线网球', 742),
 ('５月', 738),
 ('４月', 710),
 ('１６', 706),
 ('３０', 652),
 ('也是', 646),
 ('月1', 644),
 ('１７', 635),
 ('月１', 633),
 ('月２', 593),
 ('新华社欧新', 593),
 ('20162017赛季', 584),
 ('１５', 568),
 ('１２', 559),
 ('1日', 557),
 ('更多', 556),
 ('年6', 554),
 ('比0', 551),
 ('１１', 548),
 ('１９', 544),
 ('新华社法新', 529),
 ('月2', 527),
 ('新华社路透', 526),
 ('新华社北京', 524),
 ('中的', 524),
 ('欧新新华社', 515),
 ('的是', 501),
 ('的2017', 499),
 ('在20162017', 482),
 ('2日', 479),
 ('月23', 477),
 ('3月', 474),
 ('自己的', 465),
 ('23日', 463),
 ('５０', 455),
 ('网球法网', 448),
 ('

In [38]:
get_pair_prob=get_prob_from_counts(two_gram_counts)

In [55]:
##生成词汇条件概率
def get_2_gram_prob(word,prev):
    return get_pair_prob(prev+word)/get_prob(word)

def get_2_gram_strings_prob(strings):
    probability=[]
    strings_word=sentence_seg(strings)
    for m,n in enumerate(strings_word):
        prev='<s>'if m==0 else strings_word[m-1]
        probability.append(get_2_gram_prob(n,prev))
    return reduce(mul,probability)

In [56]:
get_prob_performance(get_2_gram_strings_prob, pairs)

前天晚上吃晚饭的时候出现的概率是2.5543491966351226e-07
前天晚上吃早饭的时候出现的概率是2.964506062164491e-10
正是一个好看的小猫出现的概率是7.992277658551338e-11
真是一个好看的小猫出现的概率是2.0932899101791687e-10
我无言以对，简直出现的概率是0.0003400271229370146
我简直无言以对出现的概率是0.00034002712293701453


In [54]:
get_prob_performance(get_prob, pairs)

前天晚上吃晚饭的时候出现的概率是4.041843554119873e-22
前天晚上吃早饭的时候出现的概率是2.3217538029146583e-25
正是一个好看的小猫出现的概率是1.1533767205925158e-19
真是一个好看的小猫出现的概率是2.2018228223093513e-20
我无言以对，简直出现的概率是8.713901812802775e-16
我简直无言以对出现的概率是8.713901812802775e-16
